# Matcher 
### Created By: Jacob Gross <br> Last Modified: July 12, 2020

To run, press SHIFT+ENTER on each cell. <br>
To get to next cell, press the down arrow after clicking on a blank spot of the cell. <br>
<b> Likewise, you can click on "Restart & Run All" under the Kernel menu (may need to re-run cells with dropdown boxes)</b><br>
Cells can be toggled on and off by running the cell preceding this message.
## If asked to "Try and re-run cell", then click on a blank spot of the cell until no dropdown box or button is selected and press SHIFT+ENTER

In [ ]:
# Widgets
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from ipywidgets import Layout, interact

# Imports
import pandas as pd
import numpy as np
import math

import io
import requests

import time

import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Variables
scope = "https://spreadsheets.google.com/feeds"
credentials = ServiceAccountCredentials.from_json_keyfile_name("matcher-272116-8801bce55bcb.json", scope)
gs = gspread.authorize(credentials)

# Toggle Code Button
HTML('''<script>
code_show=true;
function code_toggle() {
    if (code_show){
    $('div.input').hide();
    } else {
    $('div.input').show();
    }
    code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the code."></form>
''')

# Journey Data

In [ ]:
express_guaranteed = widgets.Dropdown(
    options = ["Select option", "Matcher Sheet", "Cyfe Board (May be empty now)"],
    value = "Select option",
    description = "Load data from:",
    style = {"description_width":"initial"})

express_guaranteed_button = widgets.Button(description = "Load", layout = Layout(width = "80px"))
out_journey = widgets.Output()

data = None

def load_journey_data(_) :
    global data, continue_load
    with out_journey :
        clear_output()
        print("Loading journeys", end = "\r")
        
        if express_guaranteed.value == "Matcher Sheet" :
            url = "https://docs.google.com/spreadsheets/d/1a2Ewf1T363tSrwSay7xDs4WMiaxqb2CwXgJzkvXX700/edit#gid=0"

            express_guaranteed_sheet = gs.open_by_url(url).worksheet("Journeys").get_all_values()
            data = pd.DataFrame(express_guaranteed_sheet[1:], columns = express_guaranteed_sheet[0])
            data = data.replace("", np.nan)
            print("{0} journeys loaded, {1} expressions    ".format(data["journey_id"].nunique(), len(data["name#reg"].dropna())))
        elif express_guaranteed.value == "Cyfe Board (May be empty now)" :
            url = "https://www.anyvan.com/board/cyfe/express_interest_expresses"
            s = requests.get(url).content

            try :
                data = pd.read_csv(io.StringIO(s.decode("utf-8"))).sort_values(by = ["start_end"], ascending = False)
                print("{0} journeys loaded, {1} expressions    ".format(data["journey_id"].nunique(), len(data["name#reg"].dropna())))
            except :
                data = None
                print("No journeys available")
        else :
            data = None
            print("Select one of the options")
                
express_guaranteed_button.on_click(load_journey_data)

widgets.VBox([widgets.HBox([express_guaranteed, express_guaranteed_button]), out_journey])

In [ ]:
if data is not None :
    journey_info = widgets.Dropdown(
        options = np.insert(np.array(data["journey_id"].sort_values().unique()), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
else :
    journey_info = widgets.Dropdown(
        options = ["Select journey", "Try to re-run cell"],
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})

journey_info_button = widgets.Button(description = "Display Journey Info", layout = Layout(width = "160px"))
out_journey_info = widgets.Output()
    
def display_journey_info(_):
    with out_journey_info:
        clear_output()
        
        if journey_info.value != "Select journey" and journey_info.value != "Try to re-run cell" and journey_info.value != None :
            data_TP = data.where(data["journey_id"] == journey_info.value).dropna(how = "all")
            journey = data_TP.iloc[0]
        
            try :
                start_time = float(journey["start_time"])
                start_hour = int(start_time * 24)
                start_min = int((start_time * 24 - start_hour) * 60)
                
                start_time = "{0:02d}:{1:02d}".format(start_hour, start_min)
            except :
                start_time = journey["start_time"]
        
            print(u"Journey ID: {0}\nTP Amount: \xa3{1}\nStart to End Distance: {2:.2f} km\nPostcode: {3}".format(journey["journey_id"], journey["tp_amount"], float(journey["start_end"]), journey["postcode"]))
            print("Start Time: {0}\nNumber of Men: {1}\nTail Lift: {2}\nMax Volume: {3:.2f} m^3\nMax Weight: {4} kg".format(start_time, journey["number_of_men"], int(journey["tail_lift"]) == 1, float(journey["max_m3"]), journey["max_kg"]))
            
            data_TP = data_TP.dropna(subset = ["name#reg"])
            print("\nNumber of Expressions: {0}".format(len(data_TP.index)))
            if len(data_TP) > 0 :
                print("Express Interest by:")
                for _,TP in data_TP.iterrows():
                    try :
                        tp_quote_return = int(TP["tp_quote_return"]) == 1
                    except :
                        tp_quote_return = False
                        
                    print("\nTP Name and Reg: {0}\nDistance to Start: {1:.2f} km\nDistance to End: {2:.2f} km\nReturn: {3}".format(TP["name#reg"], float(TP["dist_to_start"]), float(TP["dist_to_end"]), tp_quote_return))
        
journey_info_button.on_click(display_journey_info)

widgets.VBox([widgets.HBox([journey_info, journey_info_button]), out_journey_info])

# TPs

In [ ]:
TPs = None

TPs_button = widgets.Button(description = "Load TPs", layout = Layout(width = "80 px"))
out_TPs = widgets.Output()

def load_TPs(_) :
    global TPs
    with out_TPs :
        clear_output()
        print("Loading TPs", end = "\r")
        
        # Returns
        url = "https://docs.google.com/spreadsheets/d/1_syJVv1Uni8oSa9BhtAjVcl2EZYZmJn03VKg9vofcD4/edit#gid=0"

        return_journey_converter_sheet = gs.open_by_url(url).worksheet("Input").get_all_values()[15:]
        returns = pd.DataFrame(return_journey_converter_sheet[1:], columns = return_journey_converter_sheet[0])
        returns = returns.drop(columns = returns.columns[16:])
        returns["nickname"].replace("", float("NaN"), inplace = True)
        returns = returns.dropna()

        # Providers
        sizes = {"Luton Van" : 20, 
                 "LWB up to 4m" : 10.5,
                 "MWB up to 3m" : 7.5,
                 "Not applicable" : 12,
                 "XLWB up 4m+" : 14.5,
                 "SWB up to 2.4m" : 5.5,
                 "Small Van" : 8}

        url = "https://docs.google.com/spreadsheets/d/1t0L9sgsUt-BT7wJWBNZ4fNY-Bo5GNytXP1wmkejFVw0/edit#gid=1205069010"

        TP_log_providers_sheet = gs.open_by_url(url).worksheet("format2").get_all_values()[7:]
        TP_log_providers = pd.DataFrame(TP_log_providers_sheet[1:], columns = TP_log_providers_sheet[0])
        storage_TP_reg = ["AVStore #YN19FHD", "AVStore #YN19FHF", "1London #YN19FHF", "DWB91 #YN19FHD", "AggyPat #YN19FHF"]
        providers = TP_log_providers.where(np.logical_and(TP_log_providers["vehicle"] != "N/A", ~TP_log_providers["TP & REG"].isin(storage_TP_reg))).dropna(how = "all").reset_index(drop = True).drop(columns = TP_log_providers.columns[-1])
        date = providers.columns[-1]

        providers["Size"] = [sizes[vehicle] for vehicle in providers["vehicle"]]
        providers["National?"] = [int(math.ceil(55 + (100 - float(nuts2c)) / 2 + (2000 if code.endswith("N") else 0))) for nuts2c,code in zip(providers["nuts2c"], providers[date])]
        providers["1Man"] = [1 if (code.startswith("1") or code.startswith("O")) else 0 for code in providers[date]]
        providers["2Man"] = [2 if (code.startswith("2") or code.startswith("O")) else 0 for code in providers[date]]
        providers["returns"] = [1 if returns["name#reg"].str.contains(TP).any() else 0 for TP in providers["TP & REG"]]
        providers["guaranteed"] = [1 if isreturn == 1 else -1 for isreturn in providers["returns"]]
        providers["special code"] = ["2man-TL-LDN-cc" if (TwoMan == 2 and TL == "1") else "TL-LDN-cc" if (TL == "1") else "2man-LDN-cc" if (TwoMan == 2) else "LDN-cc" for TwoMan,TL in zip(providers["2Man"], providers["TL"])]
        providers["capacity"] = [16 if size >= 16 else size for size in providers["Size"]]
        
        providers = providers.sort_values(by = ["TP & REG"]).reset_index(drop = True)
        providers["index"] = providers.index

        # Expressing TPs
        columns = ["TP", "lat", "lng", "expressions", "guaranteed", "boooked"]
        expressing_TPs = pd.DataFrame(columns = columns)

        if data is not None :
            for TP in data["name#reg"].dropna().unique() :
                data_TP = data.where(data["name#reg"] == TP).dropna(how = "all")
                lat = round(float(data_TP.iloc[0]["veh_lat"]), 2)
                lng = round(float(data_TP.iloc[0]["veh_lng"]), 2)
                expressions = len(data_TP.index)
                guaranteed = 1 if (providers["TP & REG"].str.contains(TP).any() or returns["name#reg"].str.contains(TP).any()) else 0
                booked = 1 if returns["name#reg"].str.contains(TP).any() else 0
                expressing_TPs = expressing_TPs.append(pd.DataFrame([[TP, lat, lng, expressions, guaranteed, booked]], columns = columns))
        else :
            print("No journeys available, please load data")
            print("No Express Interest TPs\n")

        expressing_TPs = expressing_TPs.sort_values(by = ["TP"]).reset_index(drop = True)
        expressing_TPs["index"] = expressing_TPs.index

        # Combine TPs
        TP_namereg = np.append(providers["TP & REG"], expressing_TPs["TP"])
        TP_nickname = [name.split(" ")[0] for name in TP_namereg]
        TP_lat = np.append(providers["Lat"], expressing_TPs["lat"])
        TP_lng = np.append(providers["Lng"], expressing_TPs["lng"])
        TP_TL = np.append(providers["TL"], expressing_TPs["expressions"])
        TP_capacity = np.append(providers["Size"], np.full(len(expressing_TPs.index), np.nan))
        TP_maxkm = np.append(providers["National?"], np.full(len(expressing_TPs.index), np.nan))
        TP_1man = np.append(providers["1Man"], np.full(len(expressing_TPs.index), np.nan))
        TP_2man = np.append(providers["2Man"], np.full(len(expressing_TPs.index), np.nan))
        TP_guaranteed = np.append(providers["guaranteed"], expressing_TPs["guaranteed"])
        TP_provider = np.append(np.ones(len(providers.index), dtype = bool), np.zeros(len(expressing_TPs.index), dtype = bool))

        columns = ["TP", "nickname", "lat", "lng", "TL", "capacity", "maxkm", "1man", "2man", "guaranteed", "provider"]
        TPs = pd.DataFrame(zip(TP_namereg, TP_nickname, TP_lat, TP_lng, TP_TL, TP_capacity, TP_maxkm, TP_1man, TP_2man, TP_guaranteed, TP_provider), columns = columns)
        TPs = TPs.where(np.logical_and(TPs["guaranteed"] <= 0, ~TPs["TP"].duplicated())).dropna(how = "all").sort_values(by = ["provider", "TP"], ascending = [False, True])
        
        print("{0} TPs loaded, {1} Guaranteed TPs      ".format(len(TPs["TP"].index), len(TPs.where(TPs["guaranteed"] < 0).dropna(how = "all"))))
    
TPs_button.on_click(load_TPs)

widgets.VBox([TPs_button, out_TPs])

In [ ]:
if TPs is not None :
    TP_info = widgets.Dropdown(
        options = np.insert(np.array(TPs["TP"]), 0, "Select TP"),
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
else :
    TP_info = widgets.Dropdown(
        options = ["Select TP", "Try to re-run cell"],
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
    
TP_info_button = widgets.Button(description = "Display TP Info",layout = Layout(width = "160px"))
out_TP_info = widgets.Output()

def display_TP_info(_) :
    with out_TP_info :
        clear_output()
        
        if TP_info.value != "Select TP" and TP_info.value != "Try to re-run cell" and TP_info.value != None :
            TP = TPs.where(TPs["TP"] == TP_info.value).dropna(how = "all").iloc[0]
            print("TP Name and Reg: {0}\nTail Lift: {1}\nNumber of Men: {2}".format(TP["TP"], (int(TP["TL"]) == 1) if TP["provider"] == 1 else "Unknown", "1man" if TP["1man"] == 1 else "2man"))
            print("Vehicle Volume: {0} m^3\nMax Distance: {1} km\nGuaranteed: {2}\nProvider: {3}".format(TP["capacity"] if not np.isnan(TP["capacity"]) else "Unknown" , TP["maxkm"] if not np.isnan(TP["maxkm"]) else "Unknown", TP["guaranteed"] < 0, TP["provider"] == 1))
            
            if data is not None :
                data_TP = data.where(data["name#reg"] == TP["TP"]).dropna(how = "all")
                print("\nNumber of Expressions: {0}".format(len(data_TP.index)))
                if len(data_TP.index) > 0 :
                    print("Express Interest on:")
                    for _,journey in data_TP.sort_values(by = "journey_id").iterrows():
                        try :
                            tp_quote_return = int(journey["tp_quote_return"]) == 1
                        except :
                            tp_quote_return = False
                            
                        print("\nJourney ID: {0}\nDistance to Start: {1:.2f} km\nDistance to End: {2:.2f} km\nReturn: {3}".format(journey["journey_id"], float(journey["dist_to_start"]), float(journey["dist_to_end"]), tp_quote_return))
        
TP_info_button.on_click(display_TP_info)

widgets.VBox([widgets.HBox([TP_info, TP_info_button]), out_TP_info])

# Calculate IArray

In [ ]:
iarray_button = widgets.Button(description = "Calculate IArray", layout = Layout(width = "150px"))
out_iarray = widgets.Output()

def distance_funct(start_lat, start_lng, lat, lng) :
    return 2 * 6371 * np.arcsin(np.sqrt(np.sin((start_lat - lat) * np.pi / 360) ** 2 
                                           + np.cos(start_lat * np.pi / 180) 
                                           * np.cos(lat * np.pi / 180) 
                                           * np.sin((start_lng - lng) * np.pi / 360) ** 2))

iarray = None
journeys_data = None

def calc_iarray(_) :
    global iarray, journeys_data
    with out_iarray :
        clear_output()
        if data is not None :
            start_time = time.time()

            journeys = data["journey_id"].unique()

            columns = ["journey_id", "tp_amount", "vat", "vat_pef_multiplier"]
            journeys_data = pd.DataFrame(columns = columns)
            iarray = np.zeros((len(TPs.index), len(journeys)), dtype = float)

            vat_impact = 1000

            iarray_progress = widgets.IntProgress(
                value = 0,
                min = 0,
                max = len(journeys) - 1,
                step = 1,
                description = "Calculating",
                bar_style = "success",
                orientation = "horizontal",
                layout = Layout(width = "40%"),
                style = {"description_width":"initial"}
            )

            display(widgets.HBox([iarray_progress]))

            for i,journey_id in enumerate(journeys) :
                journey = data.where(data["journey_id"] == journey_id).dropna(how = "all")
                journeys_data = journeys_data.append(pd.DataFrame([[str(int(journey["journey_id"].iloc[0])), journey["tp_amount"].iloc[0], float(journey["tp_amount"].iloc[0]) / 6 / 1.2, 1 - np.tanh(float(journey["tp_amount"].iloc[0]) / 6 / 1.2 / vat_impact)]], columns = columns))

                for j in range(len(TPs.index)) :
                    if int(TPs["guaranteed"].iloc[j]) == -1 :
                        if TPs["TP"].iloc[j] in journey["name#reg"].values :
                            journey_subset = journey.where(journey["name#reg"] == TPs["TP"].iloc[j]).dropna(how = "all").iloc[0]
                            if float(journey_subset["min_dist"]) >= 5 :
                                iarray[j, i] = 5
                            else :
                                iarray[j, i] = float(journey_subset["min_dist"])
                        else :
                            if float(journey["max_m3"].iloc[0]) <= float(TPs["capacity"].iloc[j]) :
                                if float(journey["start_end"].iloc[0]) <= float(TPs["maxkm"].iloc[j]) :
                                    if int(journey["number_of_men"].iloc[0]) == int(TPs["1man"].iloc[j]) or int(journey["number_of_men"].iloc[0]) == int(TPs["2man"].iloc[j]) :
                                        if int(journey["tail_lift"].iloc[0]) <= int(TPs["TL"].iloc[j]) :
                                            distance = distance_funct(float(journey["start_lat"].iloc[0]), float(journey["start_lng"].iloc[0]), float(TPs["lat"].iloc[j]), float(TPs["lng"].iloc[j]))
                                            if distance <= (float(TPs["maxkm"].iloc[j]) if float(TPs["maxkm"].iloc[j]) < 2000 else float(TPs["maxkm"].iloc[j]) - 2000) :
                                                iarray[j, i] = distance
                    elif TPs["TP"].iloc[j] in journey["name#reg"].values :
                        iarray[j, i] = float(journey.where(journey["name#reg"] == TPs["TP"].iloc[j]).dropna(how = "all")["min_dist"].iloc[0])
                iarray_progress.value = i

            print("Took {0:.3f} secs to run".format(time.time() - start_time))

        else :
            print("No journeys available")
            return

        journeys_data = journeys_data.reset_index(drop = True)
    
iarray_button.on_click(calc_iarray)

widgets.VBox([iarray_button, out_iarray])

In [ ]:
if data is not None :
    journey_menu = widgets.Dropdown(
        options = np.insert(np.array(journeys_data.sort_values(by = "journey_id")["journey_id"]), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    TP_menu = widgets.Dropdown(
        options = np.insert(np.array(TPs["TP"]), 0, "Select TP"),
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
else :
    journey_menu = widgets.Dropdown(
        options = ["Select journey", "Try to re-run cell"],
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    TP_menu = widgets.Dropdown(
        options = ["Select TP", "Try to re-run cell"],
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})

journey_button = widgets.Button(description = "Find", layout = Layout(width = "80px"))
TP_button = widgets.Button(description = "Find", layout = Layout(width = "80px"))
out_display = widgets.Output()

def journey_display(_) :
    TP_menu.value = "Select TP"
    
    with out_display :
        clear_output()
        if journey_menu.value != "Select journey" and journey_menu.value != "Try to re-run cell" and journey_menu.value != None:
            print("Journey: {0}\n".format(journey_menu.value))
            for TP, iarray_val in list(zip(TPs["TP"], iarray.transpose()[np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu.value)][0][0])) :
                print("{0:20} -> {1}".format(TP, iarray_val))

def TP_display(_) :
    journey_menu.value = "Select journey"
    
    with out_display :
        clear_output()
        if TP_menu.value != "Select TP" and TP_menu.value != "Try to re-run cell" and TP_menu.value != None:
            print("TP: {0}\n".format(TP_menu.value))
            for journey, iarray_val in list(zip(journeys_data["journey_id"], iarray[np.argwhere(np.array(TPs["TP"]) == TP_menu.value)][0][0])) :
                print("{0:20} -> {1}".format(journey, iarray_val))

journey_button.on_click(journey_display)
TP_button.on_click(TP_display)

widgets.VBox([widgets.HBox([journey_menu, journey_button, TP_menu, TP_button]), out_display])

In [ ]:
if data is not None :
    journey_menu_edit = widgets.Dropdown(
        options = np.insert(np.array(journeys_data.sort_values(by = "journey_id")["journey_id"]), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    TP_menu_edit = widgets.Dropdown(
        options = np.insert(np.array(TPs["TP"]), 0, "Select TP"),
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
else :
    journey_menu_edit = widgets.Dropdown(
        options = ["Select journey", "Try to re-run cell"],
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    TP_menu_edit = widgets.Dropdown(
        options = ["Select TP", "Try to re-run cell"],
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
    
iarray_edit_box = widgets.FloatText(
    value = 0,
    description = "New IArray value:",
    style = {"description_width":"initial"})

iarray_edit_button = widgets.Button(description = "Update IArray", layout = Layout(width = "115px"))
out_edit = widgets.Output()

def iarray_edit(_) :
    with out_edit :
        if journey_menu_edit.value == "Try to re-run cell" or TP_menu_edit.value == "Try to re-run cell" :
            print("Try to re-run cell\n")
        elif journey_menu_edit.value == "Select journey" or TP_menu_edit.value == "Select TP" :
            print("Select a journey and a TP\n")
        elif iarray_edit_box.value is None or iarray_edit_box.value < 0 :
            print("Select an appropriate value for the iarray edit\n")
        else :
            old_iarray_value = iarray[np.argwhere(np.array(TPs["TP"]) == TP_menu_edit.value), np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu_edit.value)][0][0]
            iarray[np.argwhere(np.array(TPs["TP"]) == TP_menu_edit.value), np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu_edit.value)] = float(iarray_edit_box.value)
            print("TP: {0}\nJourney: {1}\nOld IArray Value: {2}\nNew Iarray Value: {3}\n".format(TP_menu_edit.value, journey_menu_edit.value, old_iarray_value, iarray_edit_box.value))
            
iarray_edit_button.on_click(iarray_edit)

widgets.VBox([widgets.HBox([journey_menu_edit, TP_menu_edit, iarray_edit_box, iarray_edit_button]), out_edit])

# Provider Stats and TP Score

In [ ]:
provider_stats_TP_score_button = widgets.Button(description = "Load Provider Stats and Calculate TP Score", layout = Layout(width = "300px"))
out_provider_stats_TP_score = widgets.Output()

TP_pstats = None

def load_stat_calc_score(_) :
    global TP_pstats
    with out_provider_stats_TP_score :
        clear_output()
        
        if TPs is None :
            print("Load TPs First")
            return
        
        print("Loading provider stats", end = "\r")
        url = "https://www.anyvan.com/board/cyfe/express_interest_pstats"
        s = requests.get(url).content

        try :
            pstats = pd.read_csv(io.StringIO(s.decode("utf-8")))
            print("Provider stats loaded                ")
        except :
            data = None
            print("No provider stats available")

        score_weight = {"rank_vat" : 10,
                        "rank_g" : 25,
                        "rank_wps" : 1,
                        "rank_rating" : 2,
                        "rank_dsw" : 1,
                        "rank_ssw" : 2,
                        "rank_exp" : 1}

        TP_pstats = TPs.merge(pstats, how = "left", left_on = "nickname", right_on = "nickname")

        TP_pstats["express"] = [10 if guaranteed == -1 else TL for guaranteed, TL in zip(TP_pstats["guaranteed"], TP_pstats["TL"])]
        TP_pstats["rank_vat"] = TP_pstats["vat?"].rank(method = "min", ascending = False) / len(TP_pstats.index)
        TP_pstats["rank_g"] = TP_pstats["guaranteed"].rank(method = "min", ascending = False) / len(TP_pstats.index)
        TP_pstats["rank_wps"] = TP_pstats["wins per session"].rank(method = "min", ascending = False) / len(TP_pstats.index)
        TP_pstats["rank_rating"] = TP_pstats["avg rating"].rank(method = "min", ascending = True) / len(TP_pstats.index)
        TP_pstats["rank_dsw"] = TP_pstats["days_since_win"].rank(method = "min", ascending = True) / len(TP_pstats.index)
        TP_pstats["rank_ssw"] = TP_pstats["sessions_since_win"].rank(method = "min", ascending = True) / len(TP_pstats.index)
        TP_pstats["rank_exp"] = TP_pstats["express"].rank(method = "min", ascending = True) / len(TP_pstats.index)

        TP_pstats["weighted_score"] = TP_pstats["rank_vat"] * score_weight["rank_vat"] + TP_pstats["rank_g"] * score_weight["rank_g"] + TP_pstats["rank_wps"] * score_weight["rank_wps"] + TP_pstats["rank_rating"] * score_weight["rank_rating"] + TP_pstats["rank_dsw"] * score_weight["rank_dsw"] + TP_pstats["rank_ssw"] * score_weight["rank_ssw"] + TP_pstats["rank_exp"] * score_weight["rank_exp"]
        weighted_score_max = TP_pstats["weighted_score"].max()

        TP_pstats["weighted_rank"] = [0.75 if guaranteed == -1 else (1.5 - weighted_score / weighted_score_max) for guaranteed, weighted_score in zip(TP_pstats["guaranteed"], TP_pstats["weighted_score"])]

        print("TP Score calculated")
        
provider_stats_TP_score_button.on_click(load_stat_calc_score)

widgets.VBox([provider_stats_TP_score_button, out_provider_stats_TP_score])

# MArray 
(Need to run again if updating IArray values)

In [ ]:
marray_button = widgets.Button(description = "Calculate MArray", layout = Layout(width = "150px"))
out_marray = widgets.Output()

marray = None

def calc_marray(_) :
    global marray
    with out_marray :
        clear_output()
        
        if iarray is None :
            print("Calculate IArray First")
            return
        
        marray = iarray.copy()

        for i in range(len(journeys_data.index)) :
            for j in range(len(TP_pstats.index)) :
                if iarray[j, i] > 0 :
                    marray[j, i] = (iarray[j,i] + (100 if int(TP_pstats["guaranteed"].iloc[j]) >= 0 else 0)) * float(TP_pstats["weighted_rank"].iloc[j]) * (float(journeys_data["vat_pef_multiplier"].iloc[i]) if int(TP_pstats["vat?"].iloc[j]) == 0 else 1)
                    
        print("MArray calculated")
                    
marray_button.on_click(calc_marray)

widgets.VBox([marray_button, out_marray])

In [ ]:
if data is not None :
    journey_menu_marray = widgets.Dropdown(
        options = np.insert(np.array(journeys_data.sort_values(by = "journey_id")["journey_id"]), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    TP_menu_marray = widgets.Dropdown(
        options = np.insert(np.array(TP_pstats["TP"]), 0, "Select TP"),
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
else :
    journey_menu_marray = widgets.Dropdown(
        options = ["Select journey", "Try to re-run cell"],
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    TP_menu_marray = widgets.Dropdown(
        options = ["Select TP", "Try to re-run cell"],
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})

journey_button_marray = widgets.Button(description = "Find", layout = Layout(width = "80px"))
TP_button_marray = widgets.Button(description = "Find", layout = Layout(width = "80px"))
out_display_marray = widgets.Output()

def journey_display_marray(_) :
    TP_menu_marray.value = "Select TP"
    
    with out_display_marray :
        clear_output()
        if journey_menu_marray.value != "Select journey" and journey_menu_marray.value != "Try to re-run cell" and journey_menu_marray.value != None:
            print("Journey: {0}\n".format(journey_menu_marray.value))
            for TP, marray_val in list(zip(TP_pstats["TP"], marray.transpose()[np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu_marray.value)][0][0])) :
                print("{0:20} -> {1}".format(TP, marray_val))

def TP_display_marray(_) :
    journey_menu_marray.value = "Select journey"
    
    with out_display_marray :
        clear_output()
        if TP_menu_marray.value != "Select TP" and TP_menu_marray.value != "Try to re-run cell" and TP_menu_marray.value != None:
            print("TP: {0}\n".format(TP_menu_marray.value))
            for journey, marray_val in list(zip(journeys_data["journey_id"], marray[np.argwhere(np.array(TPs["TP"]) == TP_menu_marray.value)][0][0])) :
                print("{0:20} -> {1}".format(journey, marray_val))

journey_button_marray.on_click(journey_display_marray)
TP_button_marray.on_click(TP_display_marray)

widgets.VBox([widgets.HBox([journey_menu_marray, journey_button_marray, TP_menu_marray, TP_button_marray]), out_display_marray])

# Matcher
(Need to run again if updating IArray values)

In [ ]:
matcher_button = widgets.Button(description = "Run Matcher", layout = Layout(width = "150px"))
out_matcher = widgets.Output()

matches = None
marray_copy = None
number_iterations = 25

def matcher(_):
    global matches, marray_copy
    with out_matcher :
        clear_output()
        
        if marray is None :
            print("Calculate MArray First")
            return
        
        matches = journeys_data.copy()
        matches["TP"] = np.empty(len(matches.index), dtype = object)
        
        marray_copy = marray.copy()
        journeys_data_copy = journeys_data.copy()
        TP_pstats_copy = TP_pstats.copy()
        
        matcher_progress = widgets.IntProgress(
            value = 0,
            min = 0,
            max = len(journeys_data_copy["journey_id"]) - 1,
            step = 1,
            description = "Calculating",
            bar_style = "success",
            orientation = "horizontal",
            layout = Layout(width = "40%"),
            style = {"description_width":"initial"}
        )

        display(widgets.HBox([matcher_progress]))
        
        for iteration in range(number_iterations) :
            matcher_progress.value = 0
            matcher_progress.max = len(journeys_data_copy["journey_id"]) - 1
            
            #print(len(journeys_data_copy.index), marray_copy.shape, len(TP_pstats_copy.index))

            for journey_id in journeys_data_copy["journey_id"] :
                journey = journeys_data_copy.where(journeys_data_copy["journey_id"] == journey_id).dropna(how = "all")
                journey_index = journeys_data_copy.index[journeys_data_copy["journey_id"] == journey_id][0]
                marray_journey = marray_copy[:,journey_index]
                
                if len(marray_journey[np.nonzero(marray_journey)]) > 0 :
                    best_TP_index = np.argwhere(marray_journey == np.min(marray_journey[np.nonzero(marray_journey)]))[0][0]

                    if len(marray_journey[np.nonzero(marray_journey)]) == 1 :
                        matches.loc[journeys_data.index[journeys_data["journey_id"] == journey_id][0], "TP"] = TP_pstats_copy.iloc[best_TP_index]["TP"]

                        journeys_data_copy = journeys_data_copy.drop(journey_index).reset_index(drop = True)
                        TP_pstats_copy = TP_pstats_copy.drop(best_TP_index).reset_index(drop = True)
                        marray_copy = np.delete(marray_copy, best_TP_index, 0)
                        marray_copy = np.delete(marray_copy, journey_index, 1)
                    else :
                        marray_TP = marray_copy[best_TP_index, :]
                        best_journey_index = np.argwhere(marray_TP == np.min(marray_TP[np.nonzero(marray_TP)]))[0][0]

                        if journey_index == best_journey_index :
                            matches.loc[journeys_data.index[journeys_data["journey_id"] == journey_id][0], "TP"] = TP_pstats_copy.iloc[best_TP_index]["TP"]

                            journeys_data_copy = journeys_data_copy.drop(journey_index).reset_index(drop = True)
                            TP_pstats_copy = TP_pstats_copy.drop(best_TP_index).reset_index(drop = True)
                            marray_copy = np.delete(marray_copy, best_TP_index, 0)
                            marray_copy = np.delete(marray_copy, journey_index, 1)

                print("{0} Journeys Unallocated, {1} Guaranteed TP(s) Left, Iteration {2} of {3}      ".format(len(journeys_data_copy), len(TP_pstats_copy.where(TP_pstats_copy["guaranteed"] < 0).dropna(how = "all").index), iteration + 1, number_iterations), end = "\r")
                matcher_progress.value += 1

matcher_button.on_click(matcher)

widgets.VBox([matcher_button, out_matcher])

In [ ]:
matcher_display_button = widgets.Button(description = "Display Results", layout = Layout(width = "150px"))
matcher_save_button = widgets.Button(description = "Save Results", layout = Layout(width = "150px"))
out_matcher_display = widgets.Output()

def matcher_display(_) :
    with out_matcher_display :
        clear_output()
        
        if matches is None :
            print("Run Matcher First")
            return
        
        print("Guaranteed TP(s) Left:")
        unmatched_TPs = TP_pstats.where(~TP_pstats["TP"].isin(matches["TP"])).dropna(how = "all")
        for TP in unmatched_TPs.where(unmatched_TPs["guaranteed"] < 0).dropna(how = "all")["TP"].sort_values() :
            print(TP)

        print("\nUnmatched Journeys:")
        unmatched_journeys = matches.where(matches["TP"].isnull()).dropna(how = "all")
        for journey_id in unmatched_journeys["journey_id"].sort_values() :
            print(journey_id)
            
        print("\n{0:15} {1:15} {2:25} {3:15} {4:15}".format("Journey ID", "Journey Min", "Provider Given", "Guaranteed", "Distance"))
        for i,match in matches.iterrows() :
            if match["TP"] is not None :
                TP_data = TP_pstats.where(TP_pstats["TP"] == match["TP"]).dropna(how = "all").iloc[0]
                TP = TP_data["TP"]
                guaranteed = "Yes" if int(TP_data["guaranteed"]) < 0 else "No"
                distance = "{:.2f}".format(iarray[np.argwhere(np.array(TP_pstats["TP"]) == TP_data["TP"])][0][0][i])
            else :
                TP = "-"
                guaranteed = "-"
                distance = "-"

            print("{0:15} {1:15} {2:25} {3:15} {4:15}".format(match["journey_id"], 
                                                       "{:.2f}".format(np.min(iarray[:,i][np.nonzero(iarray[:,i])])) if len(iarray[:,i][np.nonzero(iarray[:,i])]) > 0 else "-",
                                                       TP,
                                                       guaranteed,
                                                       distance))
            
def matcher_save(_) :
    with out_matcher_display :
        clear_output()
        
        if matches is None :
            print("Run Matcher First")
            return
        
        print("Saving Matcher Results", end = "\r")
        
        url = "https://docs.google.com/spreadsheets/d/1a2Ewf1T363tSrwSay7xDs4WMiaxqb2CwXgJzkvXX700/edit#gid=0"
        results_sheet = gs.open_by_url(url).worksheet("Results")
        results_sheet.clear()
        
        columns = ["Reg", "TP", "Journey ID", "Start Lat", "Start Lng", "Start End", "Men", "Tail Lift", "Max m3", "Journey Min", "Provider Given", "Guaranteed", "Distance", "Increase On Min"]
        matcher_output = pd.DataFrame(columns = columns)
        
        for i,match in matches.iterrows() :
            journey = data.where(data["journey_id"] == match["journey_id"]).dropna(how = "all").iloc[0]
            journey_min = np.min(iarray[:,i][np.nonzero(iarray[:,i])]) if len(iarray[:,i][np.nonzero(iarray[:,i])]) > 0 else "-"
            
            if match["TP"] is not None :
                TP_data = TP_pstats.where(TP_pstats["TP"] == match["TP"]).dropna(how = "all").iloc[0]
                TP = TP_data["TP"]
                reg = TP.split("#")[1]
                nickname = TP.split("#")[0]
                guaranteed = TP_data["guaranteed"]
                distance = iarray[np.argwhere(np.array(TP_pstats["TP"]) == TP_data["TP"])][0][0][i]
            else :
                TP = "-"
                reg = "-"
                nickname = "-"
                guaranteed = "-"
                distance = "-"
            
            matcher_output = matcher_output.append(pd.DataFrame([[reg, nickname, match["journey_id"], journey["start_lat"], journey["start_lng"], journey["start_end"], journey["number_of_men"], journey["tail_lift"], journey["max_m3"], journey_min, TP, guaranteed, distance, (distance - journey_min) if journey_min != "-" and distance != "-" else "-"]], columns = columns))
        
        results_sheet.update([matcher_output.columns.values.tolist()] + matcher_output.values.tolist())
        
        unmatched_TPs = TP_pstats.where(np.logical_and(~TP_pstats["TP"].isin(matches["TP"]), TP_pstats["guaranteed"] < 0)).dropna(how = "all")
        results_sheet.update("R1", "Left TPs")
        results_sheet.update("R2:R", [[TP] for TP in unmatched_TPs["TP"]])
        
        print("Saved Matcher Results              ")
        print("Go to: https://docs.google.com/spreadsheets/d/1a2Ewf1T363tSrwSay7xDs4WMiaxqb2CwXgJzkvXX700/edit?usp=sharing to view")
        
matcher_display_button.on_click(matcher_display)
matcher_save_button.on_click(matcher_save)
widgets.VBox([widgets.HBox([matcher_display_button, matcher_save_button]), out_matcher_display])

# Done